# IT Infrastructure Ontology - Analysis and Visualization

This notebook provides interactive analysis and visualization of the IT Infrastructure and Application Dependency Ontology.

## Features

- Load and query RDF/Turtle data
- Execute SPARQL queries
- Visualize infrastructure components
- Interactive network graphs
- Statistical analysis
- Export results

## Setup

First, install required packages:

```bash
pip install rdflib pandas matplotlib networkx pyvis plotly
```

In [ ]:
# Import required libraries
from rdflib import Graph, Namespace, RDF, RDFS
from rdflib.plugins.sparql import prepareQuery
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from pyvis.network import Network
import plotly.graph_objects as go
import plotly.express as px
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✓ Libraries imported successfully")

In [ ]:
# Define ontology namespaces
ONT = Namespace("http://example.org/it-infrastructure-ontology#")
INST = Namespace("http://example.org/instances#")

print("✓ Namespaces defined")
print(f"  Ontology: {ONT}")
print(f"  Instances: {INST}")